In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd

import scanpy as sc
import voyagerpy as vp
import seaborn as sns
import os
import pickle
from matplotlib.pyplot import imread
from collections import OrderedDict
import json

from matplotlib import pyplot as plt

In [2]:
path_016 = "/data/kanferg/Sptial_Omics/playGround/Data/Visium_HD_Mouse_Brain_square_example/square_016um"
andata016_ = sc.read_visium(path=path_016)
andata016_

/gpfs/gsfs10/users/kanferg/conda/envs/squidpy-voyagerpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gsfs10/users/kanferg/conda/envs/squidpy-voyagerpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gsfs10/users/kanferg/conda/envs/squidpy-voyagerpy/lib/python3.8/site-packages/scanpy/readwrite.py:415: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  positions = pd.read_csv(files["tissue_positions_file"], header=None)


AnnData object with n_obs × n_vars = 98917 × 19059
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [3]:
andata016_.obsm['spatial'] = np.array(andata016_.obsm['spatial'], dtype=np.float64)
sc.pp.filter_cells(andata016_, min_counts = 100)
sc.pp.filter_cells(andata016_, min_genes = 100)
andata016_

/gpfs/gsfs10/users/kanferg/conda/envs/squidpy-voyagerpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/gpfs/gsfs10/users/kanferg/conda/envs/squidpy-voyagerpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 98244 × 19059
    obs: 'in_tissue', 'array_row', 'array_col', 'n_counts', 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [4]:
andata016_ = sc.pp.subsample(andata016_, n_obs=7000,copy=True)
andata016_

AnnData object with n_obs × n_vars = 7000 × 19059
    obs: 'in_tissue', 'array_row', 'array_col', 'n_counts', 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [5]:
andata016_.uns['config'] = OrderedDict()
andata016_.uns["config"]["secondary_var_names"] = andata016_.var_names
is_mt = andata016_.var_names.str.startswith('mt')
vp.utils.add_per_cell_qcmetrics(andata016_, subsets={'mito': is_mt})
# The original count data
andata016_.layers['counts'] = andata016_.X.copy()
# Log-normalize the adata.X matrix
vp.utils.log_norm_counts(andata016_, inplace=True)
andata016_.layers['logcounts'] = andata016_.X.copy()

In [6]:
sc.pp.highly_variable_genes(andata016_, n_top_genes=1500,  layer="logcounts")

In [7]:
andata016_ = andata016_[:, andata016_.var["highly_variable"]]

In [8]:
andata016_.obsm['spatial'] = np.array(andata016_.obsm['spatial'], dtype=np.float64)
andata016_.uns['spatial']['img'] = andata016_.uns['spatial']['Visium_HD_Mouse_Brain'].pop("images")
andata016_.uns['spatial']['scale'] = andata016_.uns['spatial']['Visium_HD_Mouse_Brain'].pop("scalefactors")
andata016_.uns['spatial']['metadata'] = andata016_.uns['spatial']['Visium_HD_Mouse_Brain'].pop("metadata")
andata016_.uns['spatial'].pop("Visium_HD_Mouse_Brain")

spot_diameter_fullres = andata016_.uns['spatial']['scale'].pop('spot_diameter_fullres')
andata016_.uns['spatial']['scale']['spot_diameter_fullres'] = {'pxl_col_in_fullres':spot_diameter_fullres,'pxl_row_in_fullres':spot_diameter_fullres}
scale = 1
scale_dict = andata016_.uns["spatial"].get("scale", {})
spot_diam = scale_dict.get("spot_diameter_fullres")
visium_spots = gpd.GeoSeries.from_xy(andata016_.obsm['spatial'][:,0], andata016_.obsm['spatial'][:,1]).scale(scale, scale, origin=(0, 0))
_ = vp.spatial.set_geometry(andata016_, geom="spot_poly", values=visium_spots)

In [9]:
andata016_.X = vp.utils.scale(andata016_.X, center=True)
sc.tl.pca(andata016_, use_highly_variable=True, n_comps=30, random_state=1337)
andata016_.X = andata016_.layers['logcounts'].copy()

In [10]:
from leidenalg import ModularityVertexPartition
sc.pp.neighbors(
    andata016_,
    n_pcs=9,
    use_rep='X_pca',
    method='gauss',
    n_neighbors=80
)
sc.tl.leiden(
    andata016_,
    random_state=29,
    resolution=None,
    key_added='cluster',
    partition_type=ModularityVertexPartition
)

In [11]:
andata016_

AnnData object with n_obs × n_vars = 7000 × 1500
    obs: 'in_tissue', 'array_row', 'array_col', 'n_counts', 'n_genes', 'sum', 'detected', 'subsets_mito_sum', 'subsets_mito_detected', 'subsets_mito_percent', 'cluster'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'spatial', 'config', 'hvg', 'pca', 'neighbors', 'leiden'
    obsm: 'spatial', 'geometry', 'X_pca'
    varm: 'PCs'
    layers: 'counts', 'logcounts'
    obsp: 'distances', 'connectivities'

In [12]:
markers = vp.utils.find_markers(andata016_, hvg = False)

In [15]:
markers.keys()

dict_keys(['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9', 'cluster_10', 'cluster_11'])

In [16]:
markers

{'cluster_0':                 p.value       FDR  summary.AUC
 Hpca       2.860637e-09  0.000004     0.637403
 Sept3      1.015466e-03  0.761600     0.573147
 Agtpbp1    4.613767e-03  1.000000     0.559151
 Gng13      7.554957e-03  1.000000     0.548863
 Rasd2      1.041066e-02  1.000000     0.548751
 ...                 ...       ...          ...
 Ninj2      1.000000e+00  1.000000     0.500000
 Nme8       1.000000e+00  1.000000     0.500000
 Olfr263    1.000000e+00  1.000000     0.500000
 Hist1h2bp  1.000000e+00  1.000000     0.500000
 Sox17      1.000000e+00  1.000000     0.500000
 
 [1500 rows x 3 columns],
 'cluster_1':           p.value  FDR  summary.AUC
 Zc3h7b   0.154008  1.0     0.518234
 Arrb1    0.191950  1.0     0.515861
 Pvr      0.227670  1.0     0.504014
 Zfp747   0.233299  1.0     0.501710
 Mtbp     0.236802  1.0     0.501372
 ...           ...  ...          ...
 Ttyh2    1.000000  1.0     0.444324
 Cd300c2  1.000000  1.0     0.500000
 Cyp3a13  1.000000  1.0     0.500000


In [17]:
markers['cluster_0']

,p.value,FDR,summary.AUC
Hpca,2.860637e-09,0.000004,0.637403
Sept3,1.015466e-03,0.761600,0.573147
Agtpbp1,4.613767e-03,1.000000,0.559151
Gng13,7.554957e-03,1.000000,0.548863
Rasd2,1.041066e-02,1.000000,0.548751
...,...,...,...
Ninj2,1.000000e+00,1.000000,0.500000
Nme8,1.000000e+00,1.000000,0.500000
Olfr263,1.000000e+00,1.000000,0.500000
Hist1h2bp,1.000000e+00,1.000000,0.500000


In [18]:
marker_genes = [
    marker.sort_values(by='p.value').iloc[0].name
    for _, marker in sorted(markers.items())
]
marker_genes

['Hpca',
 'Zc3h7b',
 'Rsph1',
 'Gfpt2',
 'Mt1',
 'Camk2n1',
 'Ahnak',
 'Cercam',
 'Tfap2d',
 'Rgl3',
 'Krt28',
 'Cldn11']

In [20]:
marker_genes_symbols = andata016_.var.loc[marker_genes, "symbol"].tolist()

KeyError: 'symbol'

In [19]:
andata016_

AnnData object with n_obs × n_vars = 7000 × 1500
    obs: 'in_tissue', 'array_row', 'array_col', 'n_counts', 'n_genes', 'sum', 'detected', 'subsets_mito_sum', 'subsets_mito_detected', 'subsets_mito_percent', 'cluster'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'spatial', 'config', 'hvg', 'pca', 'neighbors', 'leiden'
    obsm: 'spatial', 'geometry', 'X_pca'
    varm: 'PCs'
    layers: 'counts', 'logcounts'
    obsp: 'distances', 'connectivities'

In [6]:
gene_var = vp.utils.model_gene_var(andata016_.layers['logcounts'], gene_names=andata016_.var_names)
hvgs = vp.utils.get_top_hvgs(gene_var)

# Set the 'highly_variable' column for the genes
adata_tissue.var['highly_variable'] = False
adata_tissue.var.loc[hvgs, 'highly_variable'] = True

ValueError: shape mismatch: value array of shape (18992,) could not be broadcast to indexing result of shape (19004,)